In [11]:
import pandas as pd

data = pd.read_csv("thanksgiving.csv", encoding="latin-1")

data.head()

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain
3,4337933040,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Homemade,NaN,...,Yes,No,No,No,NaN,Urban,30 - 44,Male,"$200,000 and up",Pacific
4,4337931983,Yes,Tofurkey,NaN,Baked,NaN,Bread-based,NaN,Canned,NaN,...,Yes,No,No,No,NaN,Urban,30 - 44,Male,"$100,000 to $124,999",Pacific


In [44]:
data.columns[:10]

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)'],
      dtype='object')

In [5]:
data["Do you celebrate Thanksgiving?"].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

### Filter out rows with Do you celebrate Thanksgiving? answer NO

In [14]:
data = data[(data["Do you celebrate Thanksgiving?"] == "Yes")]
data["Do you celebrate Thanksgiving?"].value_counts()


Yes    980
Name: Do you celebrate Thanksgiving?, dtype: int64

In [15]:
data["What is typically the main dish at your Thanksgiving dinner?"].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

### Extract rows with Tofurkey as primary dish and display the answer to "Do you typically have gravy?"

In [19]:
main_Tofurkey = (data["What is typically the main dish at your Thanksgiving dinner?"] == "Tofurkey")
data[main_Tofurkey]["Do you typically have gravy?"]

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

### Find out how many people ate one of the following pies: Apple, Pumpkin, Pecan

#### First lets create boolean series for each of the pies. After that we can combine these vectors with logical & operator. If the values at the same index in all the vectors is null than the person did not eat one of our pies.

In [36]:
apple_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"])
pumpkin_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"])
pecan_isnull = pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"])
data.shape

(980, 65)

#### Then let's create a combined boolean series. The resulting series will have a value of True for every person that DID NOT EAT one of the pies

In [34]:
ate_none_of_the_pies = (apple_isnull & pumpkin_isnull & pecan_isnull)
ate_none_of_the_pies.value_counts()

False    876
True     104
dtype: int64

### Convert age from range format (ex. 18-29) to the first valid integer (ex. 18)

#### First, let's create a function that converts the age

In [41]:
def convert_range_age_to_int(str_age):
    if pd.isnull(str_age):
        return None
    
    age = int(str_age.split(" ")[0].replace("+",""))
    
    return age   

#### Let's apply this function to the "Age" column and store the resulting series to the "int_age" column of the dataframe

In [43]:
data["int_age"] = data["Age"].apply(convert_range_age_to_int)
data["int_age"].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

### Convert income from range_str to numeric

#### Let's create a function to handle this. The income is in the format "\$10,000 to \$24,999" or "\$200,000 and up"

In [50]:
def convert_range_income_to_num(str_income):
    if pd.isnull(str_income):
        return None
    splited_str = str_income.split(" ")
    if splited_str[0] == "Prefer":
        return None
    income = int(splited_str[0].replace("$","").replace(",",""))
    return income



#### Let's apply this function to the "How much total combined money did all members of your HOUSEHOLD earn last year?" column and store the resulting series to the "int_income" column of the dataframe

In [51]:
data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(convert_range_income_to_num)
data["int_income"].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

### We can now see how the distance someone travels for Thanksgiving dinner relates to their income level.

In [64]:
income_lower_than_150 = data[data["int_income"] < 150000]
counts_lower_150 = income_lower_than_150["How far will you travel for Thanksgiving?"].value_counts()

counts_lower_150 = counts_lower_150 / counts_lower_150.sum() * 100
counts_lower_150

Thanksgiving is happening at my home--I won't travel at all                         40.783745
Thanksgiving is local--it will take place in the town I live in                     29.462990
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    21.770682
Thanksgiving is out of town and far away--I have to drive several hours or fly       7.982583
Name: How far will you travel for Thanksgiving?, dtype: float64

In [63]:
income_higher_than_150 = data[data["int_income"] >= 150000]
counts_higher_150 = income_higher_than_150["How far will you travel for Thanksgiving?"].value_counts()
counts_higher_150 = counts_higher_150 / counts_higher_150.sum() * 100
counts_higher_150

Thanksgiving is happening at my home--I won't travel at all                         47.142857
Thanksgiving is local--it will take place in the town I live in                     24.285714
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    17.857143
Thanksgiving is out of town and far away--I have to drive several hours or fly      10.714286
Name: How far will you travel for Thanksgiving?, dtype: float64

In [67]:
pivot_age = pd.pivot_table(data = data, index="Have you ever tried to meet up with hometown friends on Thanksgiving night?", columns = 'Have you ever attended a "Friendsgiving?"', values="int_age")
pivot_age

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [68]:
pivot_income = pd.pivot_table(data = data, index="Have you ever tried to meet up with hometown friends on Thanksgiving night?", columns = 'Have you ever attended a "Friendsgiving?"', values="int_income")
pivot_income

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842


Here are some potential next steps:

* Figure out the most common dessert people eat.
* Figure out the most common complete meal people eat.
* Identify how many people work on Thanksgiving.
* Find regional patterns in the dinner menus.
* Find age, gender, and income based patterns in dinner menus.